In [ ]:
import threading


def thread_hello():
    other = threading.Thread(target=thread_say_hello, args=())
    other.start()
    thread_say_hello()


def thread_say_hello():
    print('hello from', threading.current_thread().name)


thread_hello()

In [ ]:
import multiprocessing


def process_hello():
    other = multiprocessing.Process(target=process_say_hello, args=())
    other.start()
    process_say_hello()


def process_say_hello():
    print('hello from', multiprocessing.current_process().name)


process_hello()

In [ ]:
items = []
flag = []


def consume():
    while not flag:
        pass
    print('items is', items)


def produce():
    consumer = threading.Thread(target=consume, args=())
    consumer.start()
    for i in range(10):
        items.append(i)
    flag.append('go')


produce()

In [ ]:
from queue import Queue
import threading

queue = Queue()


def synchronized_consume():
    while True:
        print('got an item:', queue.get())
        queue.task_done()


def synchronized_produce():
    consumer = threading.Thread(target=synchronized_consume(), args=())
    consumer.daemon = True
    consumer.start()
    for i in range(10):
        queue.put(i)
    queue.join()


synchronized_produce()

In [ ]:
import threading

seen = set()
seen_lock = threading.Lock()


def already_seen(item):
    seen_lock.acquire()
    result = True
    if item not in seen:
        seen.add(item)
        result = False
    seen_lock.release()
    return result


def already_seen2(item):
    with seen_lock:
        if item not in seen:
            seen.add(item)
            return False
        return True

In [2]:
import threading

counters = [0, 0]
barrier = threading.Barrier(2)


def count(thread_num, steps):
    for i in range(steps):
        other = counters[1 - thread_num]
        print('reading', threading.current_thread().name, f'other = {other}')
        barrier.wait()  # wait for reads to complete
        counters[thread_num] = other + 1
        print('writing', threading.current_thread().name, f'this = {counters[thread_num]}')
        barrier.wait()  # wait for writes to complete


def threaded_count(steps):
    other = threading.Thread(target=count, args=(1, steps))
    other.start()
    count(0, steps)
    print('counters:', counters)


threaded_count(10)

readingreading MainThread other = 0
 Thread-6 other = 0
writing Thread-6 this = 1
writing MainThread this = 1
reading Thread-6 other = 1
reading MainThread other = 1
writing Thread-6 this = 2
writing MainThread this = 2
reading Thread-6 other = 2
reading MainThread other = 2
writing Thread-6 this = 3
writing MainThread this = 3
reading Thread-6 other = 3
reading MainThread other = 3
writing Thread-6 this = 4
writing MainThread this = 4
reading Thread-6 other = 4
reading MainThread other = 4
writing Thread-6 this = 5
writing MainThread this = 5
reading Thread-6 other = 5
reading MainThread other = 5
writing Thread-6 this = 6
writing MainThread this = 6
reading Thread-6 other = 6
reading MainThread other = 6
writing Thread-6 this = 7
writing MainThread this = 7
reading Thread-6 other = 7
reading MainThread other = 7
writing Thread-6 this = 8
writing MainThread this = 8
reading Thread-6 other = 8
reading MainThread other = 8
writing Thread-6 this = 9
writing MainThread this = 9
reading Th

In [1]:
import multiprocessing


def process_consume(in_pipe):
    while True:
        item = in_pipe.recv()
        if item is None:
            return
        print('got an item:', item)


def process_produce():
    pipe = multiprocessing.Pipe(False)
    consumer = multiprocessing.Process(target=process_consume, args=(pipe[0],))
    consumer.start()
    for i in range(10):
        pipe[1].send(i)
    pipe[1].send(None)  # done signal


process_produce()